<a href="https://colab.research.google.com/github/touseefyounas/PV_Analysis/blob/main/PVWatts_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import data_table
import requests
import pandas as pd
import plotly.express as px
import folium
from folium.raster_layers import ImageOverlay
import plotly.graph_objects as go


In [2]:
url="https://developer.nrel.gov/api/pvwatts/v8"
api_key="Rrm78PdXJt41jDkniLFvy5M2NyFY90cxQd2xGopJ"
latitude=33.670491
longitude=72.995602
params={'format':'json',
        'api_key':api_key,
        'system_capacity':4,
        'module_type':0,
        'losses':14,
        'array_type':1,
        'tilt':30,
        'azimuth':180,
        'lat':latitude,
        'lon':longitude,
        'albedo':0.3,
        'inv_eff':94}

response=requests.get(f"{url}.json",params=params)

if response.status_code==200:
  pv_data=response.json()

  #ac_monthly=pv_data['outputs']['ac_monthly']
  output_data=pv_data['outputs']
else:
  print(f"Error: {response.status_code} - {response.text}")

Months=['January','February','March','April','May','June','July','August','September','October','November','December']

#ac_pd=pd.DataFrame(table_data)
output_df=pd.DataFrame(output_data)
output_df.insert(0,'Month',Months)
output_df.head(5)

,Month,ac_monthly,poa_monthly,solrad_monthly,dc_monthly,ac_annual,solrad_annual,capacity_factor
0,January,398.835934,132.543368,4.275593,427.217916,5858.325266,5.774239,16.718965
1,February,392.508191,134.826148,4.815220,420.352844,5858.325266,5.774239,16.718965
2,March,532.965181,192.995428,6.225659,570.160548,5858.325266,5.774239,16.718965
3,April,529.392108,193.091504,6.436383,566.859141,5858.325266,5.774239,16.718965
4,May,585.730486,218.233952,7.039805,626.853278,5858.325266,5.774239,16.718965


In [3]:
fig = px.bar(output_df, x='Month', y='solrad_monthly',
       hover_data='solrad_monthly', color='dc_monthly',
       labels={'solrad_monthly':'Monthly Average of Daily Solar Hours (h)'},color_continuous_scale=['yellow','orange','red'],opacity=0.9)
fig.update_layout(paper_bgcolor='#18191A',plot_bgcolor='#242526',font_color='#FFFFFF',coloraxis_colorbar_title_text = 'Monthly DC array output (kWhdc)',bargap=0.7)
fig.show()

In [4]:
# Create a base map
test_map = folium.Map(location=[latitude,longitude ], zoom_start=17)  # Adjust latitude and longitude as needed

# Add markers, popups, or other map features
folium.Marker([latitude, longitude], popup='Your Location').add_to(test_map)
folium.TileLayer('cartodbdark_matter').add_to(test_map)
folium.LayerControl().add_to(test_map)
# Display the map in a Jupyter Notebook or save it as an HTML file
test_map.save('map.html')  # Save as an HTML file
test_map


In [5]:
url="https://developer.nrel.gov/api/pvwatts/v8"
api_key="Rrm78PdXJt41jDkniLFvy5M2NyFY90cxQd2xGopJ"
params2={'format':'json',
        'api_key':api_key,
        'system_capacity':4,
        'module_type':0,
        'losses':14,
        'array_type':1,
        'tilt':30,
        'azimuth':180,
        'lat':33.670491,
        'lon':72.995602,
        'timeframe':'hourly',
        'albedo':0.3,
        'inv_eff':94}

hourly_response=requests.get(f"{url}.json",params=params2)

if hourly_response.status_code==200:
  pv_hourly_data=hourly_response.json()

  #ac_monthly=pv_data['outputs']['ac_monthly']
else:
  print(f"Error: {response.status_code} - {response.text}")

hourly_dn=pd.DataFrame(pv_hourly_data['outputs']['ac_monthly'])
hourly_dn

,0
0,398.835934
1,392.508191
2,532.965181
3,529.392108
4,585.730486
5,537.359062
6,510.932044
7,493.873599
8,506.515978
9,517.200605


In [6]:
pv_hourly_data['outputs'].keys()

dict_keys(['ac_monthly', 'poa_monthly', 'solrad_monthly', 'dc_monthly', 'ac_annual', 'solrad_annual', 'capacity_factor', 'ac', 'poa', 'dn', 'dc', 'df', 'tamb', 'tcell', 'wspd', 'alb'])

In [7]:
hourly_df=pd.DataFrame({'ac':pv_hourly_data['outputs']['ac'],
                        'poa':pv_hourly_data['outputs']['poa'],
                        'dn':pv_hourly_data['outputs']['dn'],
                        'dc':pv_hourly_data['outputs']['dc'],
                        'df':pv_hourly_data['outputs']['df'],
                        'tamb':pv_hourly_data['outputs']['tamb'],
                        'tcell':pv_hourly_data['outputs']['tcell'],
                        'wspd':pv_hourly_data['outputs']['wspd'],
                        'alb':pv_hourly_data['outputs']['alb']})
hourly_df.head(5)

,ac,poa,dn,dc,df,tamb,tcell,wspd,alb
0,0.0,0.0,0.0,0.0,0.0,9.736,9.736,0.0,0.2
1,0.0,0.0,0.0,0.0,0.0,9.367,9.367,0.0,0.2
2,0.0,0.0,0.0,0.0,0.0,9.002,9.002,0.0,0.2
3,0.0,0.0,0.0,0.0,0.0,8.667,8.667,0.0,0.2
4,0.0,0.0,0.0,0.0,0.0,8.443,8.443,1.0,0.2


In [8]:
for index,_ in enumerate(hourly_df.index):
  hour=(index % 24) + 1
  julian_day=(index // 24) + 1
  week=(index // 168) + 1
  hourly_df.at[index,'hour']=hour
  hourly_df.at[index,'julianDay']=julian_day
  hourly_df.at[index,'week']=week
hourly_df.head(5)

,ac,poa,dn,dc,df,tamb,tcell,wspd,alb,hour,julianDay,week
0,0.0,0.0,0.0,0.0,0.0,9.736,9.736,0.0,0.2,1.0,1.0,1.0
1,0.0,0.0,0.0,0.0,0.0,9.367,9.367,0.0,0.2,2.0,1.0,1.0
2,0.0,0.0,0.0,0.0,0.0,9.002,9.002,0.0,0.2,3.0,1.0,1.0
3,0.0,0.0,0.0,0.0,0.0,8.667,8.667,0.0,0.2,4.0,1.0,1.0
4,0.0,0.0,0.0,0.0,0.0,8.443,8.443,1.0,0.2,5.0,1.0,1.0


In [9]:
weekly_temp_pivot = hourly_df.pivot_table(index='week', columns='hour', values='tamb')
fig = px.imshow(weekly_temp_pivot.T, color_continuous_scale='RdBu_r', origin='lower')
fig.update_layout(height=600,paper_bgcolor='#18191A',plot_bgcolor='#242526',font_color='#FFFFFF',coloraxis_colorbar_title_text = 'Temperature (C)')
fig.show()

In [10]:
hourly_temp_pivot=hourly_df.pivot_table(index='julianDay', columns='hour', values='tamb')
# Define the x, y, and z data for the 3D surface plot

x = hourly_temp_pivot.columns  # Hour of the day
y = hourly_temp_pivot.index  # Day of the year
z = hourly_temp_pivot.values  # Temperature values

fig = go.Figure(data=[go.Surface(z=z, x=x, y=y)])

# Customize the 3D plot layout
fig.update_layout(scene=dict(aspectmode="manual", aspectratio=dict(x=1, y=1, z=0.5)),scene_xaxis_title='Hour of the Day',
    scene_yaxis_title='Day of the Year',scene_zaxis_title='Temperature (C)',paper_bgcolor='#18191A',font_color='#FFFFFF',
    coloraxis_colorbar_title_text='Temparature (C)')
fig.update_layout(scene = dict(xaxis = dict(backgroundcolor="rgba(0, 0, 0,0)",gridcolor="white",showbackground=True,zerolinecolor="white",),
                               yaxis = dict(backgroundcolor="rgba(0, 0, 0,0)",gridcolor="white",showbackground=True,zerolinecolor="white"),
                               zaxis = dict(backgroundcolor="rgba(0, 0, 0,0)",gridcolor="white",showbackground=True,zerolinecolor="white")))
fig.update_xaxes(range=[0, 24], title_text='Hour of the Day')
fig.update_yaxes(range=[0, 365], title_text='Day of the Year')
fig.show()


In [11]:
cost_df=output_df
utility_rate=0.102
cost_df['monthly_bill']=round(utility_rate*cost_df['ac_monthly'],2)
cost_df['cumulative_bill']=cost_df['monthly_bill'].cumsum()
cost_df.head(12)

,Month,ac_monthly,poa_monthly,solrad_monthly,dc_monthly,ac_annual,solrad_annual,capacity_factor,monthly_bill,cumulative_bill
0,January,398.835934,132.543368,4.275593,427.217916,5858.325266,5.774239,16.718965,40.68,40.68
1,February,392.508191,134.826148,4.815220,420.352844,5858.325266,5.774239,16.718965,40.04,80.72
2,March,532.965181,192.995428,6.225659,570.160548,5858.325266,5.774239,16.718965,54.36,135.08
3,April,529.392108,193.091504,6.436383,566.859141,5858.325266,5.774239,16.718965,54.00,189.08
4,May,585.730486,218.233952,7.039805,626.853278,5858.325266,5.774239,16.718965,59.74,248.82
5,June,537.359062,200.748281,6.691609,575.626870,5858.325266,5.774239,16.718965,54.81,303.63
6,July,510.932044,190.150152,6.133876,547.785767,5858.325266,5.774239,16.718965,52.12,355.75
7,August,493.873599,182.534925,5.888223,529.581995,5858.325266,5.774239,16.718965,50.38,406.13
8,September,506.515978,186.563509,6.218784,542.074811,5858.325266,5.774239,16.718965,51.66,457.79
9,October,517.200605,186.273216,6.008813,553.220841,5858.325266,5.774239,16.718965,52.75,510.54


In [12]:
fig = px.bar(cost_df, x='Month', y='monthly_bill',
       hover_data='ac_monthly', color='ac_monthly',
       labels={'value':'Monthly Value (CAD$)'},color_continuous_scale=['yellow','orange','red'],opacity=0.9)
fig.update_layout(paper_bgcolor='#18191A',plot_bgcolor='#242526',font_color='#FFFFFF',coloraxis_colorbar_title_text = 'Monthly AC system output (kWhac)',bargap=0.7)
fig.show()

In [13]:
fig = px.area(cost_df, x='Month', y='monthly_bill',
       hover_data='ac_monthly',
       labels={'value':'Monthly Value (CAD$)'})
fig.update_layout(paper_bgcolor='#18191A',plot_bgcolor='#242526',font_color='#FFFFFF')
fig.update_traces(line=dict(color='orange', width=3))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(range=[0, max(cost_df['monthly_bill'])])
fig.show()

In [14]:
fig = px.area(cost_df, x='Month', y='cumulative_bill',hover_data='cumulative_bill',labels={'cumulative_bill':'Total Value (CAD$)'})
fig.update_layout(paper_bgcolor='#18191A',plot_bgcolor='#242526',font_color='#FFFFFF',coloraxis_showscale=False)
fig.update_traces(fillcolor='rgba(245,161,39,0.3)')
fig.update_traces(line=dict(color='yellow'))
fig.update_xaxes(showgrid=False)
fig.show()

In [15]:
pv_hourly_data['outputs'].keys()

dict_keys(['ac_monthly', 'poa_monthly', 'solrad_monthly', 'dc_monthly', 'ac_annual', 'solrad_annual', 'capacity_factor', 'ac', 'poa', 'dn', 'dc', 'df', 'tamb', 'tcell', 'wspd', 'alb'])

In [16]:
test_monthly=pd.DataFrame({'ac_monthly':pv_hourly_data['outputs']['ac_monthly'],
                        'poa_monthly':pv_hourly_data['outputs']['poa_monthly'],
                        'solrad_monthly':pv_hourly_data['outputs']['solrad_monthly'],
                        'dc_monthly':pv_hourly_data['outputs']['dc_monthly']})
test_monthly.head(5)

,ac_monthly,poa_monthly,solrad_monthly,dc_monthly
0,398.835934,132.543368,4.275593,427.217916
1,392.508191,134.826148,4.815220,420.352844
2,532.965181,192.995428,6.225659,570.160548
3,529.392108,193.091504,6.436383,566.859141
4,585.730486,218.233952,7.039805,626.853278


In [17]:
hourly_poa_pivot = hourly_df.pivot_table(index='julianDay', columns='hour', values='poa')

x = hourly_poa_pivot.columns  # Hour of the day
y = hourly_poa_pivot.index  # Day of the year
z = hourly_poa_pivot.values  # Temperature values

fig = go.Figure(data=[go.Surface(z=z, x=x, y=y)])

# Customize the 3D plot layout
fig.update_layout(scene=dict(aspectmode="manual", aspectratio=dict(x=1, y=1, z=0.5)),scene_xaxis_title='Hour of the Day',
    scene_yaxis_title='Day of the Year',scene_zaxis_title='Hourly plane of array irradiance (W/m2)',paper_bgcolor='#18191A',font_color='#FFFFFF',
    coloraxis_colorbar_title_text='Temparature (C)')
fig.update_layout(scene = dict(xaxis = dict(backgroundcolor="rgba(0, 0, 0,0)",gridcolor="white",showbackground=True,zerolinecolor="white",),
                               yaxis = dict(backgroundcolor="rgba(0, 0, 0,0)",gridcolor="white",showbackground=True,zerolinecolor="white"),
                               zaxis = dict(backgroundcolor="rgba(0, 0, 0,0)",gridcolor="white",showbackground=True,zerolinecolor="white")))
fig.update_xaxes(range=[0, 24], title_text='Hour of the Day')
fig.update_yaxes(range=[0, 365], title_text='Day of the Year')
fig.show()

In [18]:
url_openei="https://api.openei.org/utility_rates"
api_utility="xsn2kIrHm3aNAaLWf0XqmvJ8NWATgxSP1TJ0nvFu"
params_openei={
    'version':'latest',
    'format':'json',
    'api_key':api_utility,
    'limit':1,
    'detail':'full',
    'lat':40.7128,
    'lon':-74.0060
                }

utility_openei=requests.get(url_openei,params=params_openei)

if utility_openei.status_code==200:
  openei_data=utility_openei.json()
else:
  print(f"Error: {utility_openei.status_code} - {utility_openei.text}")

openei_data.keys()

dict_keys(['items'])

In [33]:
url="https://developer.nrel.gov/api/utility_rates/v3"
api_key="Rrm78PdXJt41jDkniLFvy5M2NyFY90cxQd2xGopJ"
params_v3={'format':'json',
        'api_key':api_key,
        'lat':28.5384,
        'lon':-81.3789
         }

rate_response=requests.get(f"{url}.json",params=params_v3)

if rate_response.status_code==200:
  rate_data=rate_response.json()

else:
  print(f"Error: {response.status_code} - {response.text}")

residential_rate=rate_data['outputs']['residential']
industrial_rate=rate_data['outputs']['industrial']
commercial_rate=rate_data['outputs']['commercial']

# print(f'Residential Rate: {residential_rate} $/kwh')
# print(f'Industrial Rate: {industrial_rate} $/kwh')
# print(f'Commercial Rate: {commercial_rate} $/kwh')



Residential Rate: 0.1111 $/kwh
Industrial Rate: 0.0804 $/kwh
Commercial Rate: 0.0938 $/kwh


In [48]:
def get_user_input():
    print("Please select an option:")
    print("1. Enter monthly bill value")
    print("2. Enter average bill value")
    print("3. Enter monthly consumption value")
    print("4. Enter average consumption value")
    choice = input("Your choice (1-4): ")

    value = None

    if choice == '1':
        value = input("Enter your monthly bill value ($): ")
        # Process the monthly bill value
        print(f"You entered a monthly bill value of {value}$")
    elif choice == '2':
        value = input("Enter your average bill value ($): ")
        # Process the average bill value
        print(f"You entered an average bill value of {value}$")
    elif choice == '3':
        value = input("Enter your monthly consumption value (kwh): ")
        # Process the monthly consumption value
        print(f"You entered a monthly consumption value of {value} kwh")
    elif choice == '4':
        value = input("Enter your average consumption value (kwh): ")
        # Process the average consumption value
        print(f"You entered an average consumption value of {value} kwh")
    else:
        print("Invalid choice. Please enter a number between 1 and 4.")

    return value

# Call the function and assign its return value to a variable
user_input_value = get_user_input()
print(user_input_value)


Please select an option:
1. Enter monthly bill value
2. Enter average bill value
3. Enter monthly consumption value
4. Enter average consumption value
Your choice (1-4): 2
Enter your average bill value ($): 130
You entered an average bill value of 130$
130


In [ ]:
fig = go.Figure(go.Indicator(
    mode = "number+delta",
    value = user_input_value,
    number = {'prefix': "$"},
    delta = {'position': "top", 'reference': 320},
    domain = {'x': [0, 1], 'y': [0, 1]}))

fig.update_layout(paper_bgcolor = "lightgray")

fig.show()

In [57]:
# Example data
categories = ['Category 1', 'Category 2', 'Category 3']
actual_values = [70, 50, 90]  # Actual values
max_value = max(actual_values)  # Max value for the filling tube

# Create two bars for each category: one for the actual value, one for the unfilled portion
fig = go.Figure()

for category, actual in zip(categories, actual_values):
    # Bar for the actual value
    fig.add_trace(go.Bar(
        name=category,
        x=[category], y=[actual],
        marker_color='black'
    ))

    # Bar for the unfilled portion
    fig.add_trace(go.Bar(
        name='',
        x=[category], y=[max_value - actual],
        marker_color='lightgrey',
        hoverinfo='none'  # This disables hover info for the unfilled part
    ))

# Update layout
fig.update_layout(
    barmode='stack',  # This stacks the two parts of each bar on top of each other
    title_text='Filling Tube Style Bar Chart',
    yaxis=dict(title='Value'),
    xaxis=dict(title='Category')
)

fig.show()


In [20]:
for key, value in pv_hourly_data['outputs'].items():
  if isinstance(value,list):
    print(f"Length of {key}: {len(value)}")
  else:
    print(f'{key} has no len()')

Length of ac_monthly: 12
Length of poa_monthly: 12
Length of solrad_monthly: 12
Length of dc_monthly: 12
ac_annual has no len()
solrad_annual has no len()
capacity_factor has no len()
Length of ac: 8760
Length of poa: 8760
Length of dn: 8760
Length of dc: 8760
Length of df: 8760
Length of tamb: 8760
Length of tcell: 8760
Length of wspd: 8760
Length of alb: 8760
